In [122]:
! sm-docker build -t autoaipython39docker .

....[Container] 2022/09/23 14:42:42 going inside waitForAgent

[Container] 2022/09/23 14:42:42 Waiting for agent ping
[Container] 2022/09/23 14:42:43 Waiting for DOWNLOAD_SOURCE
[Container] 2022/09/23 14:42:46 Phase is DOWNLOAD_SOURCE
[Container] 2022/09/23 14:42:46 finished waitForAgent
[Container] 2022/09/23 14:42:46 inside CopySrc
[Container] 2022/09/23 14:42:46 CODEBUILD_SRC_DIR=/codebuild/output/src161236627/src
[Container] 2022/09/23 14:42:46 finished CopySrc
[Container] 2022/09/23 14:42:46 YAML location is /codebuild/output/src161236627/src/buildspec.yml
[Container] 2022/09/23 14:42:46 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/09/23 14:42:46 Processing environment variables
[Container] 2022/09/23 14:42:46 No runtime version selected in buildspec.
[Container] 2022/09/23 14:42:46 Moving to directory /codebuild/output/src161236627/src
[Container] 2022/09/23 14:42:47 Configuring ssm agent with target id: codebuild:2f5ae13f-b7bd-4a95-a293-a401c62e87

# Deploy Auto-AI pipeline on Sagemaker

In [123]:
import datetime
import tarfile

import boto3 # AWS SDK for python. Provides low-level access to AWS services
from sagemaker import get_execution_role
import sagemaker

m_boto3 = boto3.client('sagemaker') 

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  #  Bucket is a logical unit of storage in AWS S3

print('Using bucket ' + bucket)

Using bucket sagemaker-us-east-1-849589503910


### Environment

In [124]:
# S3 prefix
prefix = "DemandResponse-autoai"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [125]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [126]:

WORK_DIRECTORY = "/root/Sagemaker-AutoAI/autoai-container/data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)


In [127]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/autoai-deploy-sagemaker:latest".format(account, region)
image = "849589503910.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-5nsd2ufwkmwe:default-1662989009075"
model = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

model.fit(data_location)

2022-09-23 14:45:32 Starting - Starting the training job...
2022-09-23 14:46:00 Starting - Preparing the instances for trainingProfilerReport-1663944332: InProgress
.........
2022-09-23 14:47:25 Downloading - Downloading input data
2022-09-23 14:47:25 Training - Training image download completed. Training in progress..Starting the training.
['/opt/ml/input/data/training/test.json', '/opt/ml/input/data/training/payload.csv', '/opt/ml/input/data/training/auto-ai-pipeline.pickle', '/opt/ml/input/data/training/demandresponseAutoAiHoldout.csv']
/opt/ml/input/data/training/test.json
/opt/ml/input/data/training/payload.csv
/opt/ml/input/data/training/auto-ai-pipeline.pickle
/opt/ml/input/data/training/demandresponseAutoAiHoldout.csv
Reading model complete.

2022-09-23 14:47:56 Uploading - Uploading generated training model
2022-09-23 14:47:56 Completed - Training job completed
Training seconds: 42
Billable seconds: 42


In [128]:
from sagemaker.predictor import csv_serializer

predictor = model.deploy(1, "ml.m4.xlarge", serializer=csv_serializer)

------!

In [ ]:
import pandas as pd
df_raw=pd.read_csv('/root/auto-ai/DemandResponseInput.csv')
features=['CUSTOMER_ID','CSTFNM','Customer Last Name','Telephone Number','Email Address','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED']

df_score=df_raw[features]



In [130]:
df_score.tail(1).values

array([[1498, 'Myrna', 'McDermott', '623-434-9340',
        'Myrna.McDermott@valentina.us', 49, 'Cupertino', 'S', 'male',
        "Bachelor's degree or more", 'Employed full-time', 67,
        'High Flyers', 1860, 5210, 0.357, False, False, False, 42760,
        92.8, False, True, False, True, 'Negative', False, True, False,
        False, False, False, 3209, 0.0, 3144.82]], dtype=object)

In [140]:
print(predictor.predict(df_score.tail(1).values))


The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


b',Probability_Yes,Probability_No,Customer_ID\n0,0.850617007489502,0.149382992510498,1498\n'


In [132]:
transform_output_folder = "batch-transform-output"
output_path = "s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=output_path,
    assemble_with="Line",
    accept="text/csv",
)

In [133]:
filename="demandresponseAutoAiHoldout.csv"
filepath=data_location+"/"+filename


In [134]:
transformer.transform(
    filepath, content_type="text/csv"
)
transformer.wait()

.............................Starting the inference server with 4 workers.
[2022-09-23 14:55:50 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2022-09-23 14:55:50 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2022-09-23 14:55:50 +0000] [10] [INFO] Using worker: sync
[2022-09-23 14:55:50 +0000] [12] [INFO] Booting worker with pid: 12
[2022-09-23 14:55:50 +0000] [13] [INFO] Booting worker with pid: 13
[2022-09-23 14:55:50 +0000] [14] [INFO] Booting worker with pid: 14
Starting the inference server with 4 workers.
[2022-09-23 14:55:50 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2022-09-23 14:55:50 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2022-09-23 14:55:50 +0000] [10] [INFO] Using worker: sync
[2022-09-23 14:55:50 +0000] [12] [INFO] Booting worker with pid: 12
[2022-09-23 14:55:50 +0000] [13] [INFO] Booting worker with pid: 13
[2022-09-23 14:55:50 +0000] [14] [INFO] Booting worker with pid: 14
[2022-09-23 14:55:50 +0000] [18] [INFO] Booting worker wi

In [136]:
s3_client = sess.boto_session.client("s3")
s3_client.download_file(
    sess.default_bucket(), "{}/demandresponseAutoAiHoldout.csv.out".format(transform_output_folder), "/tmp/demandresponseAutoAiHoldout.csv.out"
)


In [138]:
with open("/tmp/demandresponseAutoAiHoldout.csv.out") as f:
    results = f.readlines()
print("Transform results: \n{}".format("".join(results)))

Transform results: 
,Probability_Yes,Probability_No,Customer_ID
0,0.9171045640872786,0.08289543591272146,3
1,0.33880077586910073,0.6611992241308993,4



In [151]:
!tar -cgvf /root/Sagemaker-AutoAI

tar: Refusing to write archive contents to terminal (missing -f option?)
tar: Error is not recoverable: exiting now
